In [17]:
# Imports
import json
from hashlib import md5

DB_FILE = '../docs/events_db.json'
EVENT_TAGS = ['exhibition',]

def load_db():
    """Load the event database from a JSON file."""
    try:
        with open(DB_FILE, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return {}

def save_db(db):
    """Save the event database to a JSON file."""
    with open(DB_FILE, 'w') as file:
        json.dump(db, file, indent=4, default=str)

def generate_event_hash(event_details):
    """Generate a hash of the event details to detect changes."""
    event_string = json.dumps(event_details, sort_keys=True, default=str)
    return md5(event_string.encode('utf-8')).hexdigest()

def generate_unique_identifier(event_details):
    """Generate a unique identifier for an event."""
    return f"{event_details['name']}-{event_details['venue']}"

def process_event(event_details):
    """Check if an event is new or has changed and update the database accordingly."""
    db = load_db()
    site_events = db.get(event_details['venue'], {})
    
    event_id = generate_unique_identifier(event_details)
    event_hash = generate_event_hash(event_details)

    if event_id not in site_events or site_events[event_id]['hash'] != event_hash:
        print(f"Updating event: {event_details['name']}")
        site_events[event_id] = {**event_details, 'hash': event_hash}
        db[event_details['venue']] = site_events
        save_db(db)
    else:
        print(f"No changes detected for event: {event_details['name']}")

In [18]:
# Identify event details
event_name = 'Intimate Nature: Plants and Landscapes of Central California Paintings by Jeanne Rosen Sofen'
venue = 'San Francisco Botanical Gardens (Helen Crocker Russell Library)'
event_description = 'Using a variety of painting techniques, Jeanne Rosen Sofen has created a loving tribute to the beauty and diversity of Central California, from Sierra Nevada mountains to oak woodlands, to coastal hills and redwoods. A lifetime of enjoying and observing the natural world combined with years of exploring painting techniques has brought us this exciting exhibit of paintings of plants and landscapes.'
phase = 'current'
tags = ['exhibition'] + [phase] + ['free']
start_date = '2024-02-01'
end_date = '2024-05-31'
event_link = 'https://gggp.org/art-exhibit-jeanne-rosen-sofen/'

event_details = {
    'name': event_name,
    'venue': venue,
    'description': event_description,
    'tags': tags,
    'phase': phase,
    'dates': {'start': start_date, 'end': end_date},
    'links': [
        {
            'link': event_link,
            'description': 'Event Page'
        },
    ],

}

print(json.dumps(event_details, indent=4))

{
    "name": "Intimate Nature: Plants and Landscapes of Central California Paintings by Jeanne Rosen Sofen",
    "venue": "San Francisco Botanical Gardens (Helen Crocker Russell Library)",
    "description": "Using a variety of painting techniques, Jeanne Rosen Sofen has created a loving tribute to the beauty and diversity of Central California, from Sierra Nevada mountains to oak woodlands, to coastal hills and redwoods. A lifetime of enjoying and observing the natural world combined with years of exploring painting techniques has brought us this exciting exhibit of paintings of plants and landscapes.",
    "tags": [
        "exhibition",
        "current",
        "free"
    ],
    "phase": "current",
    "dates": {
        "start": "2024-02-01",
        "end": "2024-05-31"
    },
    "links": [
        {
            "link": "https://gggp.org/art-exhibit-jeanne-rosen-sofen/",
            "description": "Event Page"
        }
    ]
}


In [19]:
# Save event info
process_event(event_details)

Updating event: Intimate Nature: Plants and Landscapes of Central California Paintings by Jeanne Rosen Sofen
